# Caipora Project

__Objetivo__




 
__Data Source__

https://queimadas.dgi.inpe.br/queimadas/portal

https://ipsamazonia.org.br/

https://openaq.org/


__Data characteristics__

- Time Series;
- Geographic coordinates – Latitude/Longitude;
- Satellite Name

In [1]:
import os
import math
import unicodedata

import glob

import datetime
import gmaps

import numpy as np
import pandas as pd

import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

## Prerequisite

In [2]:
WORKDIR = os.path.abspath(os.getcwd())

## Constants

In [3]:
BRAZILIAN_STATES = [
    { "acronym": "AC", "name": "Acre" },
    { "acronym": "AL", "name": "Alagoas" },
    { "acronym": "AP", "name": "Amapá" },
    { "acronym": "AM", "name": "Amazonas" },
    { "acronym": "BA", "name": "Bahia" },
    { "acronym": "CE", "name": "Ceará" },
    { "acronym": "DF", "name": "Distrito Federal" },
    { "acronym": "ES", "name": "Espírito Santo" },
    { "acronym": "GO", "name": "Goiás" },
    { "acronym": "MA", "name": "Maranhão" },
    { "acronym": "MT", "name": "Mato Grosso" },
    { "acronym": "MS", "name": "Mato Grosso do Sul" },
    { "acronym": "MG", "name": "Minas Gerais" },
    { "acronym": "PA", "name": "Pará" },
    { "acronym": "PB", "name": "Paraíba" },
    { "acronym": "PR", "name": "Paraná" },
    { "acronym": "PE", "name": "Pernambuco" },
    { "acronym": "PI", "name": "Piauí" },
    { "acronym": "RJ", "name": "Rio de Janeiro" },
    { "acronym": "RN", "name": "Rio Grande do Norte" },
    { "acronym": "RS", "name": "Rio Grande do Sul" },
    { "acronym": "RO", "name": "Rondônia" },
    { "acronym": "RR", "name": "Roraima" },
    { "acronym": "SC", "name": "Santa Catarina" },
    { "acronym": "SP", "name": "São Paulo" },
    { "acronym": "SE", "name": "Sergipe" },
    { "acronym": "TO", "name": "Tocantins" }
]

## Get the data

### Hotspot data

__List files used in the analysis__

In [4]:
path = ''.join([WORKDIR, "/data/hotspot/**/*"]) 
hotspot_files = glob.glob(os.path.join(path, "*.csv"))

__Load into Pandas DataFrame__

In [5]:
hotspot_df = pd.concat(map(pd.read_csv, hotspot_files))
hotspot_df.head()

,datahora,satelite,pais,estado,municipio,bioma,diasemchuva,precipitacao,riscofogo,latitude,longitude,frp
0,2018/01/01 04:06:00,NPP-375D,Brasil,MARANHAO,CURURUPU,Amazonia,0.0,0.8,0.0,-1.87136,-44.78587,NaN
1,2018/01/01 04:06:00,NPP-375D,Brasil,MARANHAO,MARACACUME,Amazonia,0.0,0.1,0.1,-1.82566,-45.88670,NaN
2,2018/01/01 04:06:00,NPP-375D,Brasil,MARANHAO,BURITICUPU,Amazonia,0.0,1.1,0.1,-4.57874,-46.38660,NaN
3,2018/01/01 04:06:00,NPP-375D,Brasil,MARANHAO,PAULO RAMOS,Amazonia,0.0,1.4,0.1,-4.59554,-45.66039,NaN
4,2018/01/01 04:06:00,NPP-375D,Brasil,MARANHAO,ARAME,Amazonia,0.0,0.4,0.3,-5.21960,-46.12886,NaN


### Amazon SPI data

__List files used in the analysis__

In [6]:
path = ''.join([WORKDIR, "/data/spi/amazonia/detailed"])
spi_files = glob.glob(os.path.join(path, "*.csv"))

__Load into Pandas DataFrame__

In [7]:
spi_df = pd.concat(map(pd.read_csv, spi_files))
spi_df.head()

,Ano,Código IBGE,Município,Estado,IPS Amazônia,Ranking IPS,Necessidades Humanas Básicas,Fundamentos para o Bem-Estar,Oportunidades,Nutrição e cuidados médicos básicos,...,"Desmatamento recente 2019, 2019, 2020 (% área total do município)",Focos de calor 2020 (nª de focos/1.000 habitantes),Diversidade Partidária 2020 (% vereadores eleitos partidos diferentes),Transporte Público 2020 (nº de ônibus e micro-ônibus/1.000 habitantes),"Acesso à cultura, esporte e lazer 2018 (Categórica 1-10)",Gravidez na infância e adolescência 2019 (% de filhos de mães com até 19 anos),Trabalho Infantil 2019 (nº de famílias com ao menos 1 membro em trabalho infantil/1.000 famílias),Vulnerabilidade familiar 2019 (% de filhos de mães solteiras),Empregos ensino superior 2019 (% de empregos em relação ao total),Mulheres com empregos ensino superior 2019 (% de empregos em relação ao total)
0,2014,1100015.0,Alta Floresta D'Oeste,RO,56.594170,197.0,60.195511,57.369340,52.217659,95.154708,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,1100023.0,Ariquemes,RO,55.728511,264.0,59.059533,59.304304,48.821695,93.920755,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,1100031.0,Cabixi,RO,58.915724,90.0,74.178132,54.787265,47.781775,88.092664,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014,1100049.0,Cacoal,RO,61.717654,28.0,72.691093,60.613765,51.848105,91.956107,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,1100056.0,Cerejeiras,RO,54.593926,361.0,62.606094,62.512401,38.663282,87.810163,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Explore the data

### Describe the data I

__Hotspot__

In [8]:
hotspot_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
diasemchuva,11060285.0,7.623764,70.880709,-999.00000,1.00000,4.00000,12.000,120.00000
precipitacao,11060285.0,1.034430,3.854016,0.00000,0.00000,0.00000,0.300,203.70000
riscofogo,11060285.0,-5.310438,77.318383,-999.00000,0.40000,0.90000,1.000,1.00000
latitude,12651992.0,-7.019591,3.980039,-16.29000,-9.85041,-7.71500,-3.879,5.23000
longitude,12651992.0,-56.207436,6.320051,-73.93146,-61.00200,-55.22246,-51.210,-43.64537
frp,3390008.0,19.308732,59.705760,-3.70000,3.00000,7.20000,16.800,9722.60000


__Amazon SPI__

In [9]:
spi_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Ano,2319.0,2.017667e+03,2.868060e+00,2.014000e+03,2.014000e+03,2.018000e+03,2.021000e+03,2.021000e+03
Código IBGE,2316.0,2.288414e+06,1.365403e+06,1.100015e+06,1.502798e+06,1.713254e+06,2.109278e+06,5.108956e+06
IPS Amazônia,2316.0,5.456015e+01,3.811311e+00,4.359790e+01,5.192879e+01,5.430994e+01,5.680819e+01,7.441633e+01
Ranking IPS,2316.0,3.865000e+02,2.229051e+02,1.000000e+00,1.937500e+02,3.865000e+02,5.792500e+02,7.720000e+02
Necessidades Humanas Básicas,2319.0,6.437410e+01,6.825254e+00,4.337849e+01,5.969696e+01,6.432076e+01,6.887852e+01,8.712833e+01
...,...,...,...,...,...,...,...,...
Gravidez na infância e adolescência 2019 (% de filhos de mães com até 19 anos),772.0,2.384973e+01,6.295921e+00,5.172414e+00,1.955196e+01,2.412254e+01,2.815534e+01,4.545455e+01
Trabalho Infantil 2019 (nº de famílias com ao menos 1 membro em trabalho infantil/1.000 famílias),772.0,3.170346e+01,6.065901e+01,0.000000e+00,2.748239e+00,9.631206e+00,3.342557e+01,4.740000e+02
Vulnerabilidade familiar 2019 (% de filhos de mães solteiras),772.0,4.370381e+01,2.095850e+01,5.000000e+00,2.629215e+01,3.899752e+01,5.949844e+01,9.534161e+01
Empregos ensino superior 2019 (% de empregos em relação ao total),772.0,1.828965e+01,1.281655e+01,4.964422e+00,1.129523e+01,1.595505e+01,2.173583e+01,1.479940e+02


### Get information about data

__Hotspot__

In [10]:
hotspot_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12651992 entries, 0 to 318737
Data columns (total 12 columns):
 #   Column        Dtype  
---  ------        -----  
 0   datahora      object 
 1   satelite      object 
 2   pais          object 
 3   estado        object 
 4   municipio     object 
 5   bioma         object 
 6   diasemchuva   float64
 7   precipitacao  float64
 8   riscofogo     float64
 9   latitude      float64
 10  longitude     float64
 11  frp           float64
dtypes: float64(6), object(6)
memory usage: 1.2+ GB


__Amazon SPI__

In [11]:
spi_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2319 entries, 0 to 772
Columns: 158 entries, Ano to Mulheres com empregos ensino superior 2019 (% de empregos em relação ao total)
dtypes: float64(155), int64(1), object(2)
memory usage: 2.8+ MB


### Percentage of missing data

In [12]:
def get_pct_missing_data(dataset):
    """
    Get Percentage of missing data
    
    Attributes
    ----------
    dataset : Pandas DataFrame
    """
    total = dataset.isnull().sum().sort_values(ascending=False)
    percent = dataset.isnull().sum() / dataset.isnull().count() * 100
    percent = (round(percent, 4)).sort_values(ascending=False)

    missing_data = pd.concat([total, percent], keys=["Total", '%'], axis=1)

    return missing_data

__Hotspot__

In [13]:
get_pct_missing_data(hotspot_df)

,Total,%
frp,9261984,73.2057
diasemchuva,1591707,12.5807
precipitacao,1591707,12.5807
riscofogo,1591707,12.5807
datahora,0,0.0000
satelite,0,0.0000
pais,0,0.0000
estado,0,0.0000
municipio,0,0.0000
bioma,0,0.0000


__Amazon SPI__

In [14]:
get_pct_missing_data(spi_df)

,Total,%
Moradias com iluminação adequada 2018 (% de domicilios),1547,66.7098
Vulnerabilidade familiar 2017 (% de filhos de mães solteiras),1547,66.7098
Desmatamento acumulado 2017 (% área total do município),1547,66.7098
"Desmatamento recente 2017, 2017, 2018 (% área total do município)",1547,66.7098
Focos de calor 2018 (nª de focos/1.000 habitantes),1547,66.7098
...,...,...
Segurança pessoal,0,0.0000
Acesso ao conhecimento básico,0,0.0000
Acesso à informação e comunicação,0,0.0000
Saúde e bem-estar,0,0.0000


## Prepare the data

### Standardize Country State Names

The feature __estado__ in the two datasets are diffenrents. In _hotspot_df_ the name of state is long and spi_df is short.

__Amazon SPI__

In [15]:
def get_long_name_state(acronym):
    """
    Retrieves the long name of state related to acronym.
    """
    lnames = [x.get('name') for x in BRAZILIAN_STATES if x.get('acronym') == acronym]
    return lnames[0] if lnames else np.nan


def purge_spec_chars(word):
    """
    Remove all special characters in the word.
    """
    wv = word if word is not np.nan else ""
    nfkd_form = unicodedata.normalize("NFKD", wv) 
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

In [16]:
# Get full name of state
spi_df["Estado"] = spi_df["Estado"].apply(get_long_name_state)

# Remove special characteres
spi_df["Estado"] = spi_df["Estado"].apply(purge_spec_chars)

# Capitalize the name
spi_df["Estado"] = spi_df["Estado"].apply(lambda x: x.upper())

### Standardize Cities Names

The feature __municipio__ in the two datasets are diffenrents.

In [17]:
# Remove special characteres
spi_df["Município"] = spi_df["Município"].apply(purge_spec_chars)

# Capitalize the name
spi_df["Município"] = spi_df["Município"].apply(lambda x: x.upper())

### Parse datehour of string to date

__Hotspot__

In [18]:
hotspot_df['datahora'] = pd.to_datetime(hotspot_df['datahora'], format='%Y/%m/%d %H:%M:%S')

### Add features

__Hotspot__

In [19]:
hotspot_df["ano"] = pd.DatetimeIndex(hotspot_df['datahora']).year
# hotspot_df["mes"] = pd.DatetimeIndex(hotspot_df['datahora']).month
# hotspot_df["dia"] = pd.DatetimeIndex(hotspot_df['datahora']).day

### Fix or remove outliers (optional)

__Hotspot__

In [20]:
hotspot_df.loc[hotspot_df["riscofogo"] < 0, "riscofogo"] = 0
hotspot_df.loc[hotspot_df["diasemchuva"] < 0, "diasemchuva"] = 0
hotspot_df.loc[hotspot_df["frp"] < 0, "frp"] = 0

__Amazon SPI__

In [21]:
spi_df.dropna(subset=["Município", "Estado", "IPS Amazônia"], inplace=True)

### Fill in missing values (e.g., with zero, mean, median...) or drop their rows (or columns)

__Hotspot__

In [22]:
hotspot_df["frp"].fillna(value=0, inplace=True)
hotspot_df["riscofogo"].fillna(value=0, inplace=True)
hotspot_df["diasemchuva"].fillna(value=0, inplace=True)
hotspot_df["precipitacao"].fillna(value=0, inplace=True)

### Drop columns not used

__Hotspot__

In [23]:
hotspot_df = hotspot_df.drop(columns=["datahora", "satelite", "pais", "bioma", "latitude", "longitude"])
hotspot_df.head()

,estado,municipio,diasemchuva,precipitacao,riscofogo,frp,ano
0,MARANHAO,CURURUPU,0.0,0.8,0.0,0.0,2018
1,MARANHAO,MARACACUME,0.0,0.1,0.1,0.0,2018
2,MARANHAO,BURITICUPU,0.0,1.1,0.1,0.0,2018
3,MARANHAO,PAULO RAMOS,0.0,1.4,0.1,0.0,2018
4,MARANHAO,ARAME,0.0,0.4,0.3,0.0,2018


__Amazon SPI__

_Dimensions and their components:_

1. Necessidades Humanas Básicas
    * Nutrição e cuidados médicos básicos
    * Água e saneamento
    * Moradia
    * Segurança pessoal
2. Fundamentos para o Bem-Estar
    * Acesso ao conhecimento básico
    * Acesso à informação e comunicação
    * Saúde e bem-estar",
    * Qualidade do meio ambiente
3. Oportunidades
    * Direitos individuais
    * Liberdade individual e de escolha
    * Tolerância e inclusão
    * Acesso à educação superior

In [24]:
spi_df = spi_df[[
    "Ano", "Município","Estado","IPS Amazônia","Ranking IPS",
    "Necessidades Humanas Básicas", "Fundamentos para o Bem-Estar","Oportunidades", # 3 Dimensions
    "Nutrição e cuidados médicos básicos","Água e saneamento","Moradia","Segurança pessoal", # Necessidades Humanas Básicas components
    "Acesso ao conhecimento básico", "Acesso à informação e comunicação","Saúde e bem-estar","Qualidade do meio ambiente", # Fundamentos para o Bem-Estar components
    "Direitos individuais","Liberdade individual e de escolha","Tolerância e inclusão","Acesso à educação superior" # Oportunidades components
]]
spi_df.head()

,Ano,Município,Estado,IPS Amazônia,Ranking IPS,Necessidades Humanas Básicas,Fundamentos para o Bem-Estar,Oportunidades,Nutrição e cuidados médicos básicos,Água e saneamento,Moradia,Segurança pessoal,Acesso ao conhecimento básico,Acesso à informação e comunicação,Saúde e bem-estar,Qualidade do meio ambiente,Direitos individuais,Liberdade individual e de escolha,Tolerância e inclusão,Acesso à educação superior
0,2014,ALTA FLORESTA D'OESTE,RONDONIA,56.594170,197.0,60.195511,57.369340,52.217659,95.154708,26.725300,74.756358,44.145679,67.279579,12.246689,79.040866,70.910227,35.009508,69.313167,97.107230,7.440730
1,2014,ARIQUEMES,RONDONIA,55.728511,264.0,59.059533,59.304304,48.821695,93.920755,30.394326,83.599855,28.323196,69.307394,32.951213,79.714720,55.243890,29.002168,75.927282,76.373664,13.983665
2,2014,CABIXI,RONDONIA,58.915724,90.0,74.178132,54.787265,47.781775,88.092664,32.626943,77.918497,98.074422,75.614583,15.284836,74.519566,53.730077,32.391236,69.318097,79.090076,10.327693
3,2014,CACOAL,RONDONIA,61.717654,28.0,72.691093,60.613765,51.848105,91.956107,70.286301,88.352513,40.169451,78.123863,32.410825,74.847956,57.072414,28.673850,80.286243,80.457931,17.974396
4,2014,CEREJEIRAS,RONDONIA,54.593926,361.0,62.606094,62.512401,38.663282,87.810163,32.596208,79.234759,50.783248,73.719665,29.292791,70.785929,76.251219,28.424049,64.043321,52.550253,9.635506


### Describe the data II

__Hotspot__

In [25]:
hotspot_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
diasemchuva,12651992.0,10.617849,19.514854,0.0,0.0,3.0,10.0,120.0
precipitacao,12651992.0,0.904291,3.619731,0.0,0.0,0.0,0.2,203.7
riscofogo,12651992.0,0.613064,0.401937,0.0,0.2,0.8,1.0,1.0
frp,12651992.0,5.173633,32.066891,0.0,0.0,0.0,1.1,9722.6
ano,12651992.0,2017.520379,2.792640,2012.0,2015.0,2018.0,2020.0,2022.0


In [26]:
get_pct_missing_data(hotspot_df)

,Total,%
estado,0,0.0
municipio,0,0.0
diasemchuva,0,0.0
precipitacao,0,0.0
riscofogo,0,0.0
frp,0,0.0
ano,0,0.0


__Amazon SPI__

In [27]:
spi_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Ano,2316.0,2017.666667,2.868061,2014.000000,2014.000000,2018.000000,2021.000000,2021.000000
IPS Amazônia,2316.0,54.560151,3.811311,43.597897,51.928793,54.309938,56.808190,74.416331
Ranking IPS,2316.0,386.500000,222.905145,1.000000,193.750000,386.500000,579.250000,772.000000
Necessidades Humanas Básicas,2316.0,64.382508,6.825672,43.378488,59.720342,64.323796,68.882983,87.128332
Fundamentos para o Bem-Estar,2316.0,54.987886,5.494759,39.293879,50.947697,54.704587,58.360695,78.701385
Oportunidades,2316.0,44.310060,6.030640,21.269663,41.082509,45.198260,48.364121,70.141618
Nutrição e cuidados médicos básicos,2316.0,89.044665,5.530187,16.214906,87.492299,90.387805,92.280940,99.990615
Água e saneamento,2316.0,39.053140,15.855460,1.163647,28.615377,38.016425,48.419468,92.106201
Moradia,2316.0,76.378695,14.611324,9.720165,67.823184,78.639396,87.459554,99.504060
Segurança pessoal,2316.0,53.053531,18.696570,0.000000,39.436642,49.811377,64.307924,100.000000


In [28]:
get_pct_missing_data(spi_df)

,Total,%
Ano,0,0.0
Município,0,0.0
Tolerância e inclusão,0,0.0
Liberdade individual e de escolha,0,0.0
Direitos individuais,0,0.0
Qualidade do meio ambiente,0,0.0
Saúde e bem-estar,0,0.0
Acesso à informação e comunicação,0,0.0
Acesso ao conhecimento básico,0,0.0
Segurança pessoal,0,0.0
